# multiprocessing 이용해 좀 더 빠르게 크롤링하기

# Naver blog _ title/ date/ contents / id / comments 수집
## 홈트레이닝, 홈트 관련 트랜드 변화 확인 
광고 데이터 제거 : 홈트 | 홈트레이닝 -업체 -제공받 -지원받 -원고료 -협찬 -대가 -소정의  -서포터즈 -광고 -제작비

In [4]:
import requests
import pandas as pd

import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import urllib.request
from bs4 import BeautifulSoup

from multiprocessing import Pool # Pool import하기

In [5]:
import multiprocessing
multiprocessing.cpu_count() # 6코어 12processor 확인

12

In [2]:
# second 동안 스크롤다운 함수 정의
import datetime

def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break
        

In [3]:
def get_Data(driver):
    driver.switch_to.frame('cafe_main') # 프레임 변경
    soup = BeautifulSoup(driver.page_source, 'html.parser') # 페이지 준비

    #제목/ 닉네임/ 날짜 
    title = soup.select('h3.title_text')[0].text.strip() #h3태그에 title_text클래스
    nickname = soup.select('a.nickname')[0].text.strip() #a테그에 nickname클래스
    date = soup.select('span.date')[0].text[:10] #시간제외 날짜까지만 인덱싱하여 가져옴
    
    #내용
    try:
        content = soup.select('div.ContentRenderer')[0].text #div태그에 ContentRenderer클래스
    except:
        pass

    # 댓글 : 여러개 존재하므로 모두가져온다음 text만 뽑음 
    comments_all = soup.select('span.text_comment')
    comment_list =[]
    for comment in comments_all:
        comment = comment.text.strip()
        comment_list.append(comment)

    # 댓글 : 리스트-> 문자열로 저장
    seperator = '||'
    comments = seperator.join(comment_list)

    # 제목/ 닉네임/ 날짜 / 내용/ 댓글 데이터-> 리스트로 저장
    post = [title , nickname, date, content,comments]

    return post

In [1]:
from bs4 import BeautifulSoup
import requests
import time
from multiprocessing import Pool, Manager
search_word = "삼성" #검색어 지정
end = 300 #마지막 뉴스 지정
 
#list를 공유 하기 위해
manager = Manager()
title_list = manager.list()
 
def title_to_list(start):
    global title_list
    #url making
    url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}&start={}'.format(search_word,start)
    req = requests.get(url)
 
    #정상적인 request 확인
    if req.ok:
        html = req.text
        soup = BeautifulSoup(html,'html.parser')
    
        #뉴스제목 뽑아오기
        titles = soup.select(
            'ul.type01 > li > dl > dt > a'
        )
 
        #list에 넣어준다
        for title in titles:
            title_list.append(title['title'])
 
if __name__ == '__main__':
    start_time = time.time()
    pool = Pool(processes=12) #4개의 프로세스 동시에 작동
    pool.map(title_to_list,range(1,end,10)) #title_to_list라는 함수에 1 ~ end까지 10씩늘려가며 인자로 적용
    print(title_list)
    print("실행 시간 : %s초" % (time.time() - start_time))

In [1]:
from bs4 import BeautifulSoup
import requests
import time
search_word = "삼성" #검색어 지정
start = 1
end = 300 #마지막 뉴스 지정
title_list = []
 
if __name__ == '__main__':
    start_time = time.time()
    while 1:
        if start > end:
            break
        print(start)
 
        url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}&start={}'.format(search_word,start)
        req = requests.get(url)
 
        #정상적인 request 확인
        if req.ok:
            html = req.text
            soup = BeautifulSoup(html,'html.parser')
    
         #뉴스제목 뽑아오기
            titles = soup.select(
                'ul.type01 > li > dl > dt > a'
            )
 
            #list에 넣어준다
            for title in titles:
                title_list.append(title['title'])
        start += 10
    print(title_list)
    print("실행 시간 : %s초" % (time.time() - start_time))


1
11
21
31
41
51
61
71
81
91
101
111
121
131
141
151
161
171
181
191
201
211
221
231
241
251
261
271
281
291
[]
실행 시간 : 9.767016410827637초


In [1]:
from bs4 import BeautifulSoup
import requests
import time
from multiprocessing import Pool, Manager
search_word = "삼성" #검색어 지정
end = 10 #마지막 뉴스 지정
 
#list를 공유 하기 위해
manager = Manager()
title_list = manager.list()
 
def title_to_list(start):
    global title_list
    #url making
    url = 'https://search.naver.com/search.naver?where=article&query=%ED%99%88%ED%8A%B8%20%7C%20%ED%99%88%ED%8A%B8%EB%A0%88%EC%9D%B4%EB%8B%9D%20-%EC%97%85%EC%B2%B4%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%20-%EC%A7%80%EC%9B%90%EB%B0%9B%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%20-%ED%98%91%EC%B0%AC%20-%EB%8C%80%EA%B0%80%20-%EC%86%8C%EC%A0%95%EC%9D%98%20%20-%EC%84%9C%ED%8F%AC%ED%84%B0%EC%A6%88%20-%EA%B4%91%EA%B3%A0%20-%EC%A0%9C%EC%9E%91%EB%B9%84&ie=utf8&st=rel&date_option=99&date_from=2016.01.01&date_to=2016.01.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20160101to20160131&nso_open=1&t=0&mson=0&prdtype=0'
    req = requests.get(url)
 
    #정상적인 request 확인
    if req.ok:
        html = req.text
        soup = BeautifulSoup(html,'html.parser')
    
        #뉴스제목 뽑아오기
        titles = soup.select(
            'ul.type01 > li > dl > dt > a'
        )
 
        #list에 넣어준다
        for title in titles:
            title_list.append(title['title'])
 
if __name__ == '__main__':
    start_time = time.time()
    pool = Pool(processes=12) #4개의 프로세스 동시에 작동
    pool.map(title_to_list,range(1,end,5)) #title_to_list라는 함수에 1 ~ end까지 10씩늘려가며 인자로 적용
    print(title_list)
    print("실행 시간 : %s초" % (time.time() - start_time))

In [ ]:
url = 'https://search.naver.com/search.naver?where=article&query=%ED%99%88%ED%8A%B8%20%7C%20%ED%99%88%ED%8A%B8%EB%A0%88%EC%9D%B4%EB%8B%9D%20-%EC%97%85%EC%B2%B4%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%20-%EC%A7%80%EC%9B%90%EB%B0%9B%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%20-%ED%98%91%EC%B0%AC%20-%EB%8C%80%EA%B0%80%20-%EC%86%8C%EC%A0%95%EC%9D%98%20%20-%EC%84%9C%ED%8F%AC%ED%84%B0%EC%A6%88%20-%EA%B4%91%EA%B3%A0%20-%EC%A0%9C%EC%9E%91%EB%B9%84&ie=utf8&st=rel&date_option=99&date_from=2016.01.01&date_to=2016.01.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20160101to20160131&nso_open=1&t=0&mson=0&prdtype=0'

pool = Pool(processes=12)
results = pool.apply_async()
driver = webdriver.Chrome('./chromedriver') 
driver.get(url)

In [3]:
from fake_useragent import UserAgent #pip install fake-useragent
from time import sleep
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
from selenium import webdriver
import concurrent.futures
import urllib.request  
import requests
import time

    
urls = ["https://namu.wiki/w/%EB%B6%84%EB%A5%98:%EC%A0%95%EC%88%98", 
        'https://namu.wiki/w/%EB%B6%84%EB%A5%98:%EC%88%98', 
        'https://namu.wiki/w/%EB%B6%84%EB%A5%98:%ED%95%9C%EA%B5%AD%20%EC%95%84%EC%9D%B4%EB%8F%8C', 
        'https://namu.wiki/w/%EB%B6%84%EB%A5%98:%EA%B1%B8%EA%B7%B8%EB%A3%B9']

    
limit = 20

def get_sublist_href(url: str):
    namu_link = []
    request = requests.get(url)
    sleep(1)
    
    parsed_html = BeautifulSoup(request.text, 'html.parser')
    a_element_tags = parsed_html.find_all('div', attrs={'class' : 'test'})
    for tag in a_element_tags:
        for link in tag.find_all('a'):
            namu_link.append(url + link['href'])
    
    namu_link = namu_link[:limit]
    print('Number of site: ', len(namu_link))
    return namu_link

def do_html_crawl(url: str):
    request = requests.get(url)
    sleep(1)
    parsed_html = BeautifulSoup(request.text, 'html.parser')
    return parsed_html


if __name__ == "__main__":
    start_time = time.time()
    for url in urls:
        sub_url_list = get_sublist_href(url)
        for sub_url in sub_url_list:
            do_html_crawl(sub_url)
    print("--- elapsed time %s seconds ---" % (time.time() - start_time))

Number of site:  0
Number of site:  0
Number of site:  0
Number of site:  0
--- elapsed time 4.351112127304077 seconds ---


In [ ]:
for i in range(1,3):  # 3월부터 12월까지

  # 01. 페이지 준비(1월~ 2월 설정)
  if i == 1:
    url = 'https://search.naver.com/search.naver?where=article&query=%ED%99%88%ED%8A%B8%20%7C%20%ED%99%88%ED%8A%B8%EB%A0%88%EC%9D%B4%EB%8B%9D%20-%EC%97%85%EC%B2%B4%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%20-%EC%A7%80%EC%9B%90%EB%B0%9B%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%20-%ED%98%91%EC%B0%AC%20-%EB%8C%80%EA%B0%80%20-%EC%86%8C%EC%A0%95%EC%9D%98%20%20-%EC%84%9C%ED%8F%AC%ED%84%B0%EC%A6%88%20-%EA%B4%91%EA%B3%A0%20-%EC%A0%9C%EC%9E%91%EB%B9%84&ie=utf8&st=rel&date_option=99&date_from=2016.01.01&date_to=2016.01.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20160101to20160131&nso_open=1&t=0&mson=0&prdtype=0'
  if i == 2:
    url = 'https://search.naver.com/search.naver?where=article&query=%ED%99%88%ED%8A%B8%20%7C%20%ED%99%88%ED%8A%B8%EB%A0%88%EC%9D%B4%EB%8B%9D%20-%EC%97%85%EC%B2%B4%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%20-%EC%A7%80%EC%9B%90%EB%B0%9B%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%20-%ED%98%91%EC%B0%AC%20-%EB%8C%80%EA%B0%80%20-%EC%86%8C%EC%A0%95%EC%9D%98%20%20-%EC%84%9C%ED%8F%AC%ED%84%B0%EC%A6%88%20-%EA%B4%91%EA%B3%A0%20-%EC%A0%9C%EC%9E%91%EB%B9%84&ie=utf8&st=rel&date_option=99&date_from=2016.02.01&date_to=2016.02.29&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20160201to20160229&nso_open=1&t=0&mson=0&prdtype=0'
 
  driver = webdriver.Chrome('./chromedriver') 
  #driver.implicitly_wait(4) 
  driver.get(url)
  time.sleep(2)
  
  doScrollDown(102) # 스크롤다운 

  #02. 크롤링할 개별 포스팅 주소 가져오기 (각 월(3~12)당 포스팅된 컨텐츠 )
  article_list = driver.find_elements_by_css_selector('.api_txt_lines.total_tit')

  #03. 데이터 수집 시작
  total_data=[] # 총데이터 저장할 리스트

  # 카페 개별 페이지 접속
  for article in article_list:
    try: #제목링크없는 페이지 존재
      article.click()  # 한페이지에 접속완료
      time.sleep(5)
        
      # 드라이버 윈도우 설정
      change_tab = driver.window_handles[-1]
      driver.switch_to.window(change_tab) 

      try:
          data = get_Data(driver) # 수집
          total_data.append(data)
      except:
          pass

      driver.close() # 한페이지 수집완료

      # 다른 게시글 들어갈 준비 (必)
      change_tab = driver.window_handles[-1]
      driver.switch_to.window(change_tab)
    except:
      # 다른 게시글 들어갈 준비 (必)
      change_tab = driver.window_handles[-1]
      driver.switch_to.window(change_tab)

  #04.데이터프레임으로 저장
  df = pd.DataFrame(total_data,columns=['title','nickname','date','content','comments'])

  #05.엑셀파일로 저장
  globals()['writer_{}'.format(i)] = pd.ExcelWriter(f'./alldaycrawl/naver_cafe_conents_ssal_2015_{i}월.xlsx')
  df.to_excel(globals()['writer_{}'.format(i)],index =False)
  globals()['writer_{}'.format(i)].save()

  ##06. 드라이버 종료
  driver.close()

In [ ]:
for i in range(3,13):  # 3월부터 12월까지

  # 01. 페이지 준비(3월~ 12월 설정)
  if i == 3:
    url = 'https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.03.01&date_to=2015.03.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20150301to20150331&nso_open=1&t=0&mson=0&prdtype=0'
  if i == 4:
    url = 'https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.04.01&date_to=2015.04.30&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20150401to20150430&nso_open=1&t=0&mson=0&prdtype=0'
  if i == 5:
    url = 'https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.05.01&date_to=2015.05.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20150501to20150531&nso_open=1&t=0&mson=0&prdtype=0'
  if i == 6:
    url = 'https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.06.01&date_to=2015.06.30&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20150601to20150630&nso_open=1&t=0&mson=0&prdtype=0'
  if i == 7:    
    url = 'https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.07.01&date_to=2015.07.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20150701to20150731&nso_open=1&t=0&mson=0&prdtype=0'
  if i == 8:    
    url = 'https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.08.01&date_to=2015.08.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20150801to20150831&nso_open=1&t=0&mson=0&prdtype=0'
  if i == 9:    
    url = 'https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.09.01&date_to=2015.09.30&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20150901to20150930&nso_open=1&t=0&mson=0&prdtype=0'
  if i == 10:   
    url = 'https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.10.01&date_to=2015.10.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20151001to20151031&nso_open=1&t=0&mson=0&prdtype=0'
  if i == 11:    
    url = 'https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.11.01&date_to=2015.11.30&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20151101to20151130&nso_open=1&t=0&mson=0&prdtype=0'
  if i == 12:    
    url = 'https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.12.01&date_to=2015.12.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20151201to20151231&nso_open=1&t=0&mson=0&prdtype=0'

  driver = webdriver.Chrome('./chromedriver') 
  #driver.implicitly_wait(4) 
  driver.get(url)
  time.sleep(2)
  
  doScrollDown(102) # 스크롤다운 

  #02. 크롤링할 개별 포스팅 주소 가져오기 (각 월(3~12)당 포스팅된 컨텐츠 )
  article_list = driver.find_elements_by_css_selector('.api_txt_lines.total_tit')

  #03. 데이터 수집 시작
  total_data=[] # 총데이터 저장할 리스트

  # 카페 개별 페이지 접속
  for article in article_list:
    try: #제목링크없는 페이지 존재
      article.click()  # 한페이지에 접속완료
      time.sleep(5)
        
      # 드라이버 윈도우 설정
      change_tab = driver.window_handles[-1]
      driver.switch_to.window(change_tab) 

      try:
          data = get_Data(driver) # 수집
          total_data.append(data)
      except:
          pass

      driver.close() # 한페이지 수집완료

      # 다른 게시글 들어갈 준비 (必)
      change_tab = driver.window_handles[-1]
      driver.switch_to.window(change_tab)
    except:
      # 다른 게시글 들어갈 준비 (必)
      change_tab = driver.window_handles[-1]
      driver.switch_to.window(change_tab)

  #04.데이터프레임으로 저장
  df = pd.DataFrame(total_data,columns=['title','nickname','date','content','comments'])

  #05.엑셀파일로 저장
  globals()['writer_{}'.format(i)] = pd.ExcelWriter(f'./alldaycrawl/naver_cafe_conents_ssal_2015_{i}월.xlsx')
  df.to_excel(globals()['writer_{}'.format(i)],index =False)
  globals()['writer_{}'.format(i)].save()

  ##06. 드라이버 종료
  driver.close()